In [1]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

Mounted at /content/drive


### Specify here the NN topology and the dataframe to be loaded

In [2]:
#specify the NN topology here:
#input_layer_nodes = number_of_attributes
number_of_attributes = 10 
hidden_layer_nodes = 12
output_layer_nodes = 12
BATCH_SIZE = 10
EPOCHS = 10
#specify the dataset here:
#choose the dataframe which the NN will be trained and mo
df_number = 1
number_of_samples = 2000



attributes_array = ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92']    

###Define the model

In [3]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'), 
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

###Import the data

In [4]:
title_train = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_train_1_8000_samples.csv'
title_test =  f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/df_test_1_2000_samples.csv'
df_train_raw = pd.read_csv(title_train)
df_test_raw = pd.read_csv(title_test)
target_train = df_train_raw.pop('target')
target_test = df_test_raw.pop('target')

# Select a sub-dataframe from the main dataframe with a lower number of attributes
df_train = df_train_raw[attributes_array[0:number_of_attributes]]
df_test = df_test_raw[attributes_array[0:number_of_attributes]]

df_train = np.asarray(df_train).astype(np.float32)
target_train = np.asarray(target_train).astype(np.float32)
df_test_np = np.asarray(df_test).astype(np.float32)
target_test_np = np.asarray(target_test).astype(np.float32)

### NN training
 model weights file generation

 model parameters json file generation

In [5]:
# Begin NN training 
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_train)

# Execute Trainning and Testing
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
model.fit(df_train, target_train, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# model weights file generation
title_model_save = f'/content/drive/MyDrive/network-traffic-classification-main/tf-models/model_with_{number_of_attributes}_attributes_{hidden_layer_nodes}_hidden_nodes.csv'
model.save(title_model_save) 

test_loss, one_test_acc =  model.evaluate(df_test_np,  target_test_np, verbose=1, batch_size=BATCH_SIZE)      
tf_predictions_probabilities = model.predict(df_test_np)
tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)  
  j_max = x.argmax()
  tf_predictions.append(j_max)  

conf_m = tf.math.confusion_matrix(target_test_np,tf_predictions)
print(conf_m)

# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
# Report file content
report = {
    "datetime": dt_string,
    "attributes": attributes_array[0:number_of_attributes],
    "number_of_attributes": number_of_attributes,
    "hidden_layer_nodes": hidden_layer_nodes,
    "output_layer_nodes": output_layer_nodes,
    "accuracy_test": one_test_acc,
    "attributes_array": attributes_array,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "number_of_samples": number_of_samples,
    "df_number": df_number
    
}
# Writes into the file
# model parameters json file generation
title = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters_%s"%dt_string + "_with_%d_attributes.json"%number_of_attributes
with open(title, "w") as f:
  json.dump(report, f)

title_parameters_save = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/nn_model_parameters.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)







Starting training for 10 attributes
Epoch 1/10
9600/9600 [==============================] - 14s 1ms/step - loss: 0.9966 - accuracy: 0.6997
Epoch 2/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.6789 - accuracy: 0.7957
Epoch 3/10
9600/9600 [==============================] - 14s 1ms/step - loss: 0.5874 - accuracy: 0.8223
Epoch 4/10
9600/9600 [==============================] - 14s 1ms/step - loss: 0.5290 - accuracy: 0.8412
Epoch 5/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.4857 - accuracy: 0.8531
Epoch 6/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.4575 - accuracy: 0.8579
Epoch 7/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.4409 - accuracy: 0.8626
Epoch 8/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.4296 - accuracy: 0.8651
Epoch 9/10
9600/9600 [==============================] - 13s 1ms/step - loss: 0.4192 - accuracy: 0.8677
Epoch 10/10
2400/2400 [==============